<a href="https://colab.research.google.com/github/Ayushi1722/LLM-Guided-Feature-Extraction-for-HateSpeech-Classification/blob/main/train_eval_naive_hatebert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers==4.23.1
! pip install torch torchvision
! pip install tqdm

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AdamW
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset

In [3]:
print("CUDA Available: ", torch.cuda.is_available())

CUDA Available:  True


In [4]:
torch.cuda.empty_cache()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda', index=0)

In [6]:
## set dataset here
# options = ["gab", "twitter", "reddit", "youtube"]


dataset = "youtube"

file_map = {
    "gab": '/content/Rationales_file_GAB_dataset_corrected.csv',
    "twitter": '/content/Rationales_file_TWITTER_dataset.csv',
    "reddit": '/content/Rationales_file_REDDIT_dataset.csv',
    "youtube": '/content/Rationales_file_YOUTUBE_dataset.csv'
}

In [7]:
file_path = file_map[dataset]
df = pd.read_csv(file_path)
train_df = df[df['exp_split'] == 'train']
test_df = df[df['exp_split'] == 'test']

In [8]:
print("Train df: ", len(train_df))
print("Test_df: ", len(test_df))

Train df:  1024
Test_df:  4028


In [9]:
import gc
# del variables
gc.collect()

30

In [10]:
model_name = 'GroNLP/HateBERT'
tokenizer = BertTokenizer.from_pretrained("GroNLP/hateBERT")
model = BertForSequenceClassification.from_pretrained("GroNLP/hateBERT")

Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should 

In [11]:
class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    texts = self.texts[idx]
    labels = self.labels[idx]
    encoding = self.tokenizer(texts, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
    # input_ids, mask_ids = torch.tensor(encoding['input_ids']), torch.tensor(encoding['attention_mask'])
    input_ids = encoding['input_ids'].squeeze()
    attention_mask = encoding['attention_mask'].squeeze()
    labels = labels
    return input_ids, attention_mask, labels

In [12]:
# Hyperparameters for tuning model initially. Let's see, we will change if required.
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
#Splitting training and validation testing split to test accuracy
train_text, val_texts, train_labels, val_labels = train_test_split(train_df['text'].tolist(),train_df['label'].tolist(), test_size = 0.2, random_state = 42)
train_dataset = CustomDataset(train_text, train_labels, tokenizer, max_length = 512)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length = 512)

#Creating dataloader object to train the model
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=True)

In [14]:
model = model.to(device)

In [15]:
import gc
# del variables
gc.collect()

52

In [15]:
from tqdm import tqdm

num_epochs = 3
for epoch in range(num_epochs):
    model.train()

    train_losses = []
    train_accuracy = 0
    train_epoch_size = 0

    with tqdm(train_dataloader, desc=f'Epoch {epoch + 1}', dynamic_ncols=True) as loop:
      for batch in loop:
          input_ids, mask_ids, labels = batch
          if torch.cuda.is_available():
              input_ids = input_ids.to(device)
              mask_ids = mask_ids.to(device)
              labels = labels.to(device)
          # optimizer.grad()
          optimizer.zero_grad()
          outputs = model(input_ids=input_ids, attention_mask=mask_ids, labels=labels)
          loss = outputs.loss
          loss.backward()
          optimizer.step()

          train_losses.append(loss.item())

          # Update accuracy and epoch size
          predictions = torch.argmax(outputs.logits, dim=1)
          train_accuracy += (predictions == labels).sum().item()
          train_epoch_size += len(labels)

          # Update tqdm progress bar with set_postfix
          loop.set_postfix(loss=loss.item(), accuracy=train_accuracy / train_epoch_size)


    # Evaluating on Validation task
    model.eval()

    val_predictions = []
    val_labels = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, mask_ids, labels = batch
            if torch.cuda.is_available():
                input_ids = input_ids.to(device)
                mask_ids = mask_ids.to(device)
                labels = labels.to(device)
            outputs = model(input_ids=input_ids, attention_mask=mask_ids)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            val_predictions.extend(predictions.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(val_predictions, val_labels)
    print(f"Epoch {epoch + 1}: Validation Accuracy: {accuracy:.4f}")

Epoch 1: 100%|██████████| 410/410 [01:27<00:00,  4.68it/s, accuracy=0.756, loss=2.22]


Epoch 1: Validation Accuracy: 0.8293


Epoch 2: 100%|██████████| 410/410 [01:29<00:00,  4.58it/s, accuracy=0.945, loss=0.0149]


Epoch 2: Validation Accuracy: 0.8341


Epoch 3: 100%|██████████| 410/410 [01:29<00:00,  4.57it/s, accuracy=0.974, loss=0.00323]


Epoch 3: Validation Accuracy: 0.8683


In [16]:
torch.save(model, f'fine_tuned_naive_hatebert_{dataset}.pt')

# Testing on Tests Dataset for accuracy

In [17]:
test_texts = test_df['text'].tolist()
test_labels = test_df['label'].tolist()

In [18]:
test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_length = 512)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=True)

In [19]:
model.eval()
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, mask_ids, labels = batch
        if torch.cuda.is_available():
                input_ids = input_ids.to(device)
                mask_ids = mask_ids.to(device)
                labels = labels.to(device)
        outputs = model(input_ids=input_ids, attention_mask=mask_ids)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(test_predictions, test_labels)

In [20]:
print("Accuracy of test dataset:", accuracy)

Accuracy of test dataset: 0.692154915590864
